In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [2]:
path ='/Users/fyenne/Downloads/booooks/semester5/R_ML/Mar_NCAA/sb'

In [3]:
tourney_result = pd.read_csv(path+'/train_w_stg2.csv')
test_df = pd.read_csv(path+'/test_w_stg2.csv')

In [4]:
# train_list=list(tourney_result.columns)[:-1]
# tourney_result.columns[-1]

In [5]:
tourney_result = tourney_result.rename(columns = {'train_label_my':'result'})

In [6]:
import h2o
h2o.init(
  nthreads=-1,            ## -1: use all available threads
  max_mem_size = "8G")
train_list=list(tourney_result.columns)[:-1]
train_1=h2o.H2OFrame(tourney_result)
train_1['result']=train_1['result'].asfactor()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.
Parse progress: |█████████████████████████████████████████████████████████| 100%


H2O_cluster_uptime:,52 mins 01 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 17 days
H2O_cluster_name:,H2O_from_python_fyenne_2smjm1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.476 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
param = {
      "ntrees" : 2000
    , "max_depth" : 20
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4240
    , "score_tree_interval": 100
    ,"stopping_metric" :"MSE"
    ,"nfolds":8
    ,"fold_assignment":"AUTO"
    ,"keep_cross_validation_predictions" : True
    ,"booster":"dart"
}

In [9]:
from h2o.estimators import H2OXGBoostEstimator
model_xgb = H2OXGBoostEstimator(**param)
model_xgb.train(x = train_list, y = 'result', training_frame = train_1)
param={
    "ntrees" : 1060
    , "max_depth" : 3
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4240
    , "score_tree_interval": 100
    , "stopping_metric" :"MSE"
    , "nfolds":8
    , "fold_assignment":"AUTO"
    , "keep_cross_validation_predictions" : True
 }

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [10]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator  # import gbm estimator
model_gbm = H2OGradientBoostingEstimator(**param)
model_gbm.train(x = train_list, y = 'result', training_frame = train_1)

/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


gbm Model Build progress: |███████████████████████████████████████████████| 100%
stackedensemble Model Build progress: |███████████████████████████████████| 100%


In [ ]:
from h2o.estimators import H2OStackedEnsembleEstimator
stack = H2OStackedEnsembleEstimator(model_id="ensemble11",
                                       training_frame=train_1,
                                       #validation_frame=test,
                                       base_models=[model_xgb.model_id,model_gbm.model_id],
                                    metalearner_algorithm="glm")
stack.train(x=train_list, y="result", training_frame=train_1)

In [11]:
test_1=h2o.H2OFrame(test_df)
pred1=model_xgb.predict(test_1)
pred2=model_gbm.predict(test_1)
pred_df1=pred1.as_data_frame()
pred_df2=pred2.as_data_frame()

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [12]:
ids = pd.read_csv(path + '/WSampleSubmissionStage2.csv').ID.values

weights = {
    'lgb_model': 9,
    'xgb_model': 1
}

final_preds = (pred2 * weights['lgb_model'] + pred1 * weights['xgb_model']) / (weights['lgb_model'] + weights['xgb_model'])

In [13]:
final_df = pd.DataFrame(columns=['ID', 'Pred'])
final_df.ID = ids

In [16]:
final_df.Pred = h2o.as_list(final_preds).yes
final_df.to_csv('stg2_oman.csv', columns=['ID','Pred'], index=None)

In [2]:
h2o.as_list(final_preds).yes

NameError: name 'h2o' is not defined

In [3]:
!pip install tinyms==0.1.0

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/01/c6/052003531b0971d8c116212d72006fb03fd3c76220eee58fc801724cd0ec/tinyms-0.1.0-py3-none-any.whl (79 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/f2/28/2a03252dfb9ebf377f40fba6a7841b47083260bf8bd8e737b0c6952df83f/Flask-1.1.2-py2.py3-none-any.whl (94 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/4c/c5/5757886c4f538c1b3f95f6745499a24bffa389a805dee92d093e2d9ba7db/easydict-1.9.tar.gz (6.4 kB)
ERROR: Could not find a version that satisfies the requirement mindspore==1.1.1 (from tinyms)
ERROR: No matching distribution found for mindspore==1.1.1
